<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install PyPDF2
!pip install dask

In [10]:
import requests
import os
from google.colab import output
import pprint
from io import BytesIO
import PyPDF2
import re
import datetime
import dask

In [29]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [30]:
def list_endpoints():
    date_today = datetime.datetime.now()
    year_today = date_today.year
    month_today = date_today.month
    day_today = date_today.day
    key = "qu2TgXAQPvswtEZ176PFWyfvdb5wd1nwcv2JwEUo"

    endpoints = []
    for i in range(1, 3650):
        past_date = date_today - datetime.timedelta(days=i)
        year = past_date.strftime("%Y")
        month = past_date.strftime("%m")
        day = past_date.strftime("%d")
        endpoint = f"https://api.congress.gov/v3/congressional-record/?y={year}&m={month}&d={day}&api_key={key}"
        endpoints.append(endpoint)
    return endpoints


endpoints_result = list_endpoints()

In [41]:
def find_pdf_links(endpoints_list):
    response_objects = []
    for i in endpoints_list:
        response = requests.get(i)
        response_objects.append(response)
    return response_objects


delayed_pdf_links = dask.delayed(find_pdf_links)(endpoints_result)
#actual_pdf_links = dask.compute(delayed_pdf_links)
#display(actual_pdf_links)
delayed_pdf_links

Delayed('find_pdf_links-7ea86b43-c874-4e2d-8f6e-091b7d72ad17')

In [42]:
x = []
pdf_links = delayed_pdf_links.compute()


AttributeError: 'bytes' object has no attribute 'json'

In [48]:
for i in pdf_links:
  print(i.json())

{'Results': {'IndexStart': 1, 'Issues': [{'Congress': '118', 'Id': 27401, 'Issue': '89', 'Links': {'Digest': {'Label': 'Daily Digest', 'Ordinal': 1, 'PDF': [{'Part': '1', 'Url': 'https://www.congress.gov/118/crec/2024/05/22/170/89/CREC-2024-05-22-dailydigest.pdf'}]}, 'FullRecord': {'Label': 'Entire Issue', 'Ordinal': 5, 'PDF': [{'Part': '1', 'Url': 'https://www.congress.gov/118/crec/2024/05/22/170/89/CREC-2024-05-22.pdf'}]}, 'House': {'Label': 'House Section', 'Ordinal': 3, 'PDF': [{'Part': '1', 'Url': 'https://www.congress.gov/118/crec/2024/05/22/170/89/CREC-2024-05-22-house.pdf'}]}, 'Remarks': {'Label': 'Extensions of Remarks Section', 'Ordinal': 4, 'PDF': [{'Part': '1', 'Url': 'https://www.congress.gov/118/crec/2024/05/22/170/89/CREC-2024-05-22-extensions.pdf'}]}, 'Senate': {'Label': 'Senate Section', 'Ordinal': 2, 'PDF': [{'Part': '1', 'Url': 'https://www.congress.gov/118/crec/2024/05/22/170/89/CREC-2024-05-22-senate.pdf'}]}}, 'PublishDate': '2024-05-22', 'Session': '2', 'Volume': 

In [28]:
def read_pdf(response_object_list):
  total_matches = []
  for i in response_object_list[0]:
    print(i.compute().json())
    if i.compute().json()['Results']['Issues'] != []:
      link_to_pdf_file = i.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
      pdf_file = requests.get(link_to_pdf_file)
      pdf_data = BytesIO(pdf_file.content)
      pdf_reader = PyPDF2.PdfReader(pdf_data)
      number_of_pages = len(pdf_reader.pages)
      for j in range(number_of_pages):
        page = pdf_reader.pages[j]
        text = page.extract_text()
        adam_smith_matches = re.findall(r'Adam\w*\W*\sSmith\w*\W*', text)
        if adam_smith_matches != []:
          print(adam_smith_matches)
          print(link_to_pdf_file)
  total_matches += adam_smith_matches
  return total_matches

matches = read_pdf(actual_pdf_links)


{'error': {'code': 'OVER_RATE_LIMIT', 'message': 'You have exceeded your rate limit. Try again later or contact us at https://api.congress.gov:443/contact/ for assistance'}}


KeyError: 'Results'